article : https://arxiv.org/pdf/1307.0048.pdf

In [2]:
import numpy as np
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz

p = 10
n = 100
cov = toeplitz(0.5 ** np.arange(p))
A = multivariate_normal(np.zeros(p), cov, 100)
A_sc = sc.parallelize(A)

# Algorithm 1

TODO: Problème avec le calcul de covariance

In [59]:
def PenalizedLR_MR(Data, k, lambdas):
    """
    Data: an RDD each rows of which is a tuple (x, y)
    k: number of partitions for splitting
    lambdas: list of lambdas to test on
    """
    key = np.random.randint(k)
    
    def map_statistics(row):
        #calculate statistics for one row
        x, y = row
        x = x.reshape((1, -1))
        return (np.random.randint(k), [1, x, y, y**2, y * x, x.T.dot(x)])
    
    statistics = Data.map(map_statistics)
    
    def reduce_statistics(row1, row2):
        s_1 = row1[0]
        s_2 = row2[0]
        mean_x = s_1 / (s_1 + s_2) * row1[1] + s_2 / (s_1 + s_2) * row2[1]
        mean_y = s_1 / (s_1 + s_2) * row1[2] + s_2 / (s_1 + s_2) * row2[2]
        cov = s_1 / (s_1 + s_2) * row1[5] + s_2 / (s_1 + s_2) * row2[5]  + s_1 * s_2 / (s_1 + s_2)**2 * (row1[1] - row2[1]).T.dot(row1[1] - row2[1])
        emit = [s_1 + s_2, mean_x, mean_y, row1[3] + row2[3], row1[4] + row2[4], cov]
        return emit
    
    statistics = statistics.reduceByKey(reduce_statistics)
    return statistics

In [111]:
test_data = sc.parallelize([(np.array([i]), i) for i in range(1, 3)])
test_data.take(4)

[(array([1]), 1), (array([2]), 2)]

In [112]:
stat = PenalizedLR_MR(test_data, 1, [1])
stat.collect()

[(0, [2, array([[1.5]]), 1.5, 5, array([[5]]), array([[2.75]])])]

In [124]:
np.cov(np.arange(1,3), bias=True)

array(0.25)